# *Field line tracing example*
***

Example of tracing field lines

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import sunpy
import sunpy.sun.constants

In [ ]:
import pysmsh
import pysmsh.difference.staggered_curvilinear as curvilinear

In [ ]:
import cider

In [ ]:
RSun = sunpy.sun.constants.radius

### Load magnetogram & process

Load a HMI synoptic magnetogram

In [ ]:
import cider.magnetogram.hmi

In [ ]:
import os
import urllib.request

sample = 'hmi.synoptic_mr_polfil_720s.2155.Mr_polfil.fits'

if not os.path.exists('../data/'+sample):
    urllib.request.urlretrieve('https://a3s.fi/cider_sample_data/'+sample, '../data/'+sample)

In [ ]:
magnetogram_file = "../data/hmi.synoptic_mr_polfil_720s.2155.Mr_polfil.fits"

In [ ]:
raw_magnetogram = cider.magnetogram.hmi.read_hmi_synoptic(magnetogram_file)

Remap to a uniform lon-lat grid which is required by the PFSS solver

In [ ]:
import cider.utils.map
import cider.magnetogram

In [ ]:
# Create an empty map of the requested resolution
uniform_map \
    = cider.utils.map.create_full_sun_plate_carree_map(raw_magnetogram,
                                                       deg_per_pixel=1.0,
                                                       frame=raw_magnetogram.coordinate_frame.name)

In [ ]:
remapped_magnetogram = cider.magnetogram.regrid_to_grid_of_map(raw_magnetogram, uniform_map)

Balance the signed flux of the magnetogram

In [ ]:
balanced_magnetogram = cider.magnetogram.Balance.multiplicative(remapped_magnetogram)

In [ ]:
balanced_magnetogram.peek(vmin=-50, vmax=50, norm=None)

### Compute the PFSS model

In [ ]:
import cider.models.pfss

Specify the radial grid edge coordinates. For the PFSS model, the outer radius defines the source surface

In [ ]:
r = np.linspace(1.0, 2.5, 128+1)*RSun

In [ ]:
# Instantiate the model
pfss = cider.models.pfss.PotentialFieldSourceSurfaceModel(balanced_magnetogram, r)

In [ ]:
# Compute the solution
pfss.compute()

In [ ]:
# Compute the magnetic field
magnetic_field = pfss.magnetic_field()

### Tracing field lines

Tracing field lines requires 
 1. A method to compute (interpolate) the vector field at any arbitrary point from the grid values
 2. Integrating the ODE defining a streamline, $\frac{d \mathbf{x}(s)}{ds} \times \mathbf{b}(\mathbf{x}(s)) = 0$

Many different methodologies could be employed for both tasks, with varying effects on the accuracy of the computed stream line. For applications requiring a high level of accuracy, care should be taken in choosing these.

As a simple approach, we here use trilinear interpolation of the magnetic field components that have been averaged to the nodes (corners) of the mesh.

In [ ]:
import cider.utils.interpolation
import cider.utils.tracer

Average data to nodes

In [ ]:
nodal_magnetic_field = cider.utils.interpolation.average_face_staggered_to_nodal(magnetic_field)

Create the interpolator

In [ ]:
interpolator = cider.utils.interpolation.NodalSphericalShellVectorFieldInterpolator(nodal_magnetic_field)

Initialize the streamline tracer. Default parameters, which may or may not be suitable for your particular case, are set by the constructor.

In [ ]:
tracer = cider.utils.tracer.SphericalShellMidPointStreamLineTracer(interpolator)

In [ ]:
# As an example, modify the maximum length of a stream line
tracer.max_path_length = 3.0*RSun.si.value

Trace the streamline starting from a given point (given in spherical $(r, \theta, \phi$) coordinates), either in the direction of the field (reverse=False) or opposite to the direction of the field (reverse=True). The tracing is performed until it is terminated, and the end point of the streamline is returned.

In [ ]:
beg_pt = (2.0*RSun.si.value, 0.5*np.pi, 0.5*np.pi)
reverse = False

In [ ]:
end_pt = tracer.compute(beg_pt, reverse)

In [ ]:
end_pt

By default, the coordinates of the streamline are not recorded due to performance degredation when doing so. Recording of points can be switched on:

In [ ]:
tracer.record_points = True

In [ ]:
# Retrace
end_pt = tracer.compute(beg_pt, reverse)

In [ ]:
# Each coordinate is now stored in tracer.points
list(tracer.points[0:5])

Using the tracer, for example the open and closed regions can be computed.

In [ ]:
import cider.analysis.open_closed

In [ ]:
# Switch off recording of points
tracer.record_points = False

In [ ]:
%%time
open_closed_map \
    = cider.analysis.open_closed.compute_open_closed_map(pfss.magnetogram,
                                                         deg_per_pixel=2.0,
                                                         start_radius=1.01*RSun,
                                                         open_radius=2.45*RSun,
                                                         tracer=tracer)

In [ ]:
open_closed_map.peek(cmap='bwr')

The open field can be plotted together with the magnetogram by applying a mask:

In [ ]:
masked_data = np.copy(open_closed_map.data)
masked_data[np.where(np.abs(open_closed_map.data) < 1.5)] = np.nan

In [ ]:
masked_open_closed_map = sunpy.map.Map(masked_data, open_closed_map.meta)

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(projection=pfss.magnetogram)

pfss.magnetogram.plot(axes=ax, vmin=-20, vmax=20)

# Note: this uses autoalign to align the two maps of different resolution..
masked_open_closed_map.plot(axes=ax, autoalign=True, alpha=0.75, cmap="bwr", norm=None)

The PFSS result can be output to VTK for external visualization

In [ ]:
import os
import pyevtk.hl as evtk

In [ ]:
B = nodal_magnetic_field

evtk.gridToVTK("../data/" + os.path.basename(magnetogram_file).replace('.fits', '.pfss'),
               B.mesh.edges.r/RSun.si.value,
               B.mesh.edges.clt,
               B.mesh.edges.lon,
               pointData={"Br" : B.data[0], "Bt" : B.data[1], "Bp" : B.data[2]})

***